In [1]:
%%configure -f

{"conf":
     {"spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.pyspark.virtualenv.bin.path":"/home/hadoop/epidemics-venv/bin",
      "spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.dynamicAllocation.enabled":"true",
      "spark.executor.memory":"16g",
      "spark.driver.memory":"16g",
      "spark.executor.cores":"6",
      "spark.driver.cores":"6",
      "spark.driver.maxResultSize":"20g",
      "livy.server.session.timeout-check" :"false"}
}

In [2]:
import daphme as dm
import pandas as pd
from pyproj import Transformer

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

from sedona.spark import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1702682215465_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Example1").\
    getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Intro to DAPHME

In this script, we will demonstrate some of DAPHME's functionalities for analyzing human mobility data. Our analysis will focus on the month of February for the city of Philadelphia. Using a dataset that originally contains [60000] users. 

In our analysis, we will perform three different kinds of operations, which are common in the analyses in these types of data. 
* **Importing**, which optionally converts the datetime to a specified timezone and project the coordinates into ESPG:3857 (which has units in meters and might be appropriate for a local analysis not too far from the equator). 
* **Coarse Filtering**, where we subset to users to make computations for tractable. In particular, we filter out users with very few days/hours with activity, and keep TODO [users with "sufficient" pings in the area]. We also filter to users with activity in a given time period.
* **Persisting**

## Load Pings

Daphme can work with a single file in a number of formats (csv, parquet, txt) but its true utility comes when working with partitions of the data, which are often organized in a directory. In this example, we specify the path of the files to be loaded.

In [ ]:
gravy_df = spark.read.parquet("s3://phl-pings/gravy/")

gravy_df = gravy_df. \
    withColumn("latitude", F.col("geohashlatitude").cast(DoubleType())). \
    withColumn("longitude", F.col("geohashlongitude").cast(DoubleType()))

In [ ]:
cuebiq_df = spark.read.options(header='True', inferSchema='True', delimiter=',').parquet("s3://phl-pings/cuebiq-jan-mar/")

In [ ]:
cuebiq_df.show()

In [20]:
gravy_df = gravy_df.withColumn(
    "timestamp1",
    F.from_utc_timestamp(
        F.to_timestamp(F.col("timestamp") / 1000), # divide by 1000 because given in milliseconds
    F.col("timezone") # converts timestamp from UTC to local timezone
))

# gravy_df = gravy_df.withColumn(
#    "timestamp",
#    F.from_utc_timestamp(
#        F.to_utc_timestamp(gravy_df.timestamp1, F.col("timezone")), # converts ts from tz to UTC
#    F.col("timezone")) # converts ts from UTC to tz

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
gravy_df.createOrReplaceTempView("gravy_df")

gravy_df = spark.sql(
    "SELECT *, ST_MakePoint(latitude, longitude) as coordinate from gravy_df"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
gravy_df.createOrReplaceTempView("gravy_df")

# ST_Tranform uses lon/lat as its input and output
gravy_df= spark.sql(
    "SELECT *, ST_FlipCoordinates(ST_Transform(ST_FlipCoordinates(coordinate), 'EPSG:4326','EPSG:3857')) as mercator_coord from gravy_df"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
gravy_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+----------------+-----------+-------------------+----------------+--------------+------------+----------+-----------+---------+----------+--------------------+--------------------+
|                grid|geohashlatitude|geohashlongitude|geohashnine|          timestamp|        timezone|     ipaddress|forensicflag|devicetype|recordcount| latitude| longitude|          coordinate|      mercator_coord|
+--------------------+---------------+----------------+-----------+-------------------+----------------+--------------+------------+----------+-----------+---------+----------+--------------------+--------------------+
|00000006-23aa-3e4...|      39.947813|      -74.911673|  dr4er26bz|2020-03-01 16:50:37|America/New_York| 73.76.110.202|       33536|      AAID|          1|39.947813|-74.911673|POINT (39.947813 ...|POINT (4858361.50...|
|00000013-ec62-3ef...|      40.013387|      -75.144017|  dr4edh1yc|2019-10-16 05:36:06|America/New_York|  172.56.28.49|     

In [27]:
gravy_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- grid: string (nullable = true)
 |-- geohashlatitude: string (nullable = true)
 |-- geohashlongitude: string (nullable = true)
 |-- geohashnine: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timezone: string (nullable = true)
 |-- ipaddress: string (nullable = true)
 |-- forensicflag: long (nullable = true)
 |-- devicetype: string (nullable = true)
 |-- recordcount: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- coordinate: geometry (nullable = true)
 |-- mercator_coord: geometry (nullable = true)

In [7]:
testDF = gravy_df.limit(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Coarse Filtering

Takes in a geometry and filters to pings within the given geometry